# Reverse Question Generator

This is a demo of the Okareo reverse question generator. The reverse question generator is useful when you would like to automatically create test cases for retrieval models.

The generator can take documents as inputs and generate questions where the answers are in said documents.

For example, suppose that we have the following `document`:

```The quick brown fox jumps over the lazy dog.```

Then the reverse question generator might produce questions like the following:

```What type of fox jumps over the dog?```

```What does the fox do?```

```How would you describe the dog?```

To showcase the reverse question generator, we load a subset of the [Game Recommendations on Steam dataset (Kaggle)](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam) and pick a few representative samples.

In [1]:
import pandas as pd

filename = 'data/games_desc.csv'
df = pd.read_csv(filename)

N_data = 10
data_desc = df['description'].iloc[:N_data].reset_index(drop=True)
data_desc.head()

0    Enter the dark underworld of Prince of Persia ...
1    Monaco: What's Yours Is Mine is a single playe...
2    Escape Dead Island is a Survival-Mystery adven...
3    Dungeon of the Endless is a Rogue-Like Dungeon...
4    “METAL SLUG 3”, the masterpiece in SNK’s emble...
Name: description, dtype: object

Set up Okareo object by loading .env file with API key.

In [7]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetGenerate, SeedData, ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

Add the seed scenario using product descriptions as inputs, return results, and show link to UI.

In [8]:
scenario_set_create = ScenarioSetCreate(name="Reverse Question Generator: Seed Scenario",
                                        generation_type=ScenarioType.SEED,
                                        number_examples=N_data,
                                        seed_data=[SeedData(input_=data_desc.iloc[i], result="example result") for i in range(N_data)]
                                        )

source_scenario = okareo.create_scenario_set(scenario_set_create)

print(source_scenario.app_link)

https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/scenario/ed04ae35-34ae-41c9-a316-e8046498568b


Use the reverse question seed scenario to generate two examples per each point in the seed scenario, and show link to UI.

In [10]:
number_examples = 2 # number of examples to generate for each example in the seed scenario

generated_scenario = okareo.generate_scenario_set(
    ScenarioSetGenerate(
        source_scenario_id=source_scenario.scenario_id,
        name="Reverse Question Example: Generated Scenario",
        number_examples=number_examples,
        generation_type=ScenarioType.TEXT_REVERSE_QUESTION
    )
)
print(generated_scenario.app_link)

'https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/scenario/5df6612b-18f9-433c-a442-76a2c2023ae4'

Compare each generated question to the corresponding source description. 

In [11]:
# compare the descriptions
generated_points = okareo.get_scenario_data_points(generated_scenario.scenario_id)

for i in range(N_data):
    print("-"*8 + f"Example {i}" + "-"*8)
    print(f"Source: {data_desc[i]}")
    for j in range(number_examples):
        print(f"Generated #{j}: {generated_points[number_examples*i+j].input_}")
    

--------Example 0--------
Source: Enter the dark underworld of Prince of Persia Warrior Within, the sword-slashing sequel to the critically acclaimed Prince of Persia: The Sands of Time™. Hunted by Dahaka, an immortal incarnation of Fate seeking divine retribution, the Prince embarks upon a path of both carnage and mystery to defy his preordained death.
Generated #0: Can you tell me which game is the sequel to the Prince of Persia: The Sands of Time™?
Generated #1: Who is the Prince pursued by in Prince of Persia Warrior Within?
--------Example 1--------
Source: Monaco: What's Yours Is Mine is a single player or co-op heist game. Assemble a crack team of thieves, case the joint, and pull off the perfect heist.
Generated #0: Could you tell me the type of game that Monaco: What's Yours Is Mine is renowned for?
Generated #1: In the video game Monaco: What's Yours Is Mine, what is the core objective?
--------Example 2--------
Source: Escape Dead Island is a Survival-Mystery adventure that 